<a href="https://colab.research.google.com/github/niapollard/Benchmarking_QC/blob/main/052024_Al2_Varies%20Reps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q jarvis-tools qiskit==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
!pip install qiskit_nature==0.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 28.4 MB/s eta 0:00:00


In [3]:
!pip install pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 11.1 MB/s eta 0:00:00


In [4]:
!pip install qiskit-aer==0.12.0

In [5]:
import numpy as np

# Importing Qiskit and Jarvis libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from pyscf import gto, scf, tools

<ipython-input-5-ccff2813ef8f>:29: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
<ipython-input-5-ccff2813ef8f>:29: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver


In [6]:
#Build the molecule
molecule = Molecule(
    geometry=[['Al',[14.46097199, -11.1250000, 0.00000000]],
              ['Al',[17.61662800, -11.1250000, 0.00000000]]],
    charge=0,
    multiplicity=1)

In [7]:
def get_qubit_op(
    molecule,
    basis='sto3g',
    functional='lda',
    method= MethodType.RKS,
    driver_type= ElectronicStructureDriverType.PYSCF,
    converter= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]
  mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='', reps=1):
  seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['sto3g']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
#converter=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1, 2, 3, 4]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          #for n in converter:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

#print(mem)

Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7abdc88046d0> lda statevector_simulator 1


<ipython-input-7-84e3d1675a19>:96: NatureDeprecationWarning: The qiskit_nature.problems.second_quantization.electronic.ElectronicStructureProblem class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.problems.ElectronicStructureProblem class.
  res1=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k)
<ipython-input-7-84e3d1675a19>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-7-84e3d1675a19>:59: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-7-84e3d1675a19>:62: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)


...

(8.5066017987e-08+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -506.138938521234
  - computed part:      -1.99151806427
  - ActiveSpaceTransformer extracted energy part: -504.147420456964
~ Nuclear repulsion energy (Hartree): 28.339891408341
> Total ground state energy (Hartree): -477.799047112893
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 3.000 S: 0.500 S^2: 0.750 M: 0.500
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [788.03242329  -546.60328153  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [757.72351746  -525.58022283  -0.0012793]
    - computed part:      [90.9268516  -63.06975385  -0.0012793]
    - ActiveSpaceTransformer extracted energy part: [666.79666586  -462.51046899  0.0]
  > Dipole moment (a.u.): [30.30890583  -21.0230587  0.0012793]  Total: 36.88629517
                 (debye): [77.03754716  -53.43528021  0.00325165]  Total: 93.75560172
 
Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 

<ipython-input-7-84e3d1675a19>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-7-84e3d1675a19>:59: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-7-84e3d1675a19>:62: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -506.138938521749
  - computed part:      -1.991518064785
  - ActiveSpaceTransformer extracted energy part: -504.147420456964
~ Nuclear repulsion energy (Hartree): 28.339891408341
> Total ground state energy (Hartree): -477.799047113408
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 3.000 S: 0.500 S^2: 0.750 M: 0.500
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [788.03242329  -546.60328153  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [757.72351781  -525.5789164  0.00015138]
    - computed part:      [90.92685195  -63.06844741  0.00015138]
    - ActiveSpaceTransformer extracted energy part: [666.79666586  -462.51046899  0.0]
  > Dipole moment (a.u.): [30.30890548  -21.02436513  -0.00015138]  Total: 36.88703947
                 (debye): [77.03754628  -53.43860083  -0.00038476]  Total: 93.75749354
 
